In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install recbole

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00


In [3]:
!pip install modin[ray]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.

In [4]:
!pip install kmeans-pytorch

In [5]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
interactions_df = pd.read_csv('/content/drive/MyDrive/itmo_recsys/kion_dataset/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/drive/MyDrive/itmo_recsys/kion_dataset/users_processed_kion.csv')
items_df = pd.read_csv('/content/drive/MyDrive/itmo_recsys/kion_dataset/items_processed_kion.csv')

In [8]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [9]:
interactions_df['user_id'].nunique()

962179

In [10]:
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,t_dat,timestamp
0,176549,9506,2021-05-11,4250,72,2021-05-11,1620691200
1,699317,1659,2021-05-29,8317,100,2021-05-29,1622246400
2,656683,7107,2021-05-09,10,0,2021-05-09,1620518400
3,864613,7638,2021-07-05,14483,100,2021-07-05,1625443200
4,964868,9506,2021-04-30,6725,100,2021-04-30,1619740800


In [23]:
interactions_df['item_id'].nunique()

15706

In [11]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [12]:
df.head()

,user_id:token,item_id:token,timestamp:float
0,176549,9506,1620691200
1,699317,1659,1622246400
2,656683,7107,1620518400
3,864613,7638,1625443200
4,964868,9506,1619740800


In [13]:
!mkdir recbox_data

In [14]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [15]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

In [16]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [1, 0, 9]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [17]:
dataset = create_dataset(config)
logger.info(dataset)

In [18]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [19]:
import time

In [28]:
# %%time
# model_list = ['MultiVAE', 'MultiDAE', "RecVAE"]

# for model_name in model_list:
#     print(f"running {model_name}...")
#     start = time.time()
#     result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
#     t = time.time() - start
#     print(f"It took {t/60:.2f} mins")
#     print(result)

In [20]:
# result = run_recbole(model='MultiVAE', dataset = 'recbox_data',config_dict = parameter_dict )

In [22]:
from recbole.model.general_recommender.multivae import MultiVAE

In [23]:
model = MultiVAE(config, train_data.dataset).to(config['device'])

Max value of user's history interaction records has reached 20.9471766848816% of the total.


In [24]:
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

In [20]:
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]

In [21]:
len(external_user_ids)

13354

In [46]:
dataset

recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']

In [30]:
from recbole.quick_start import load_data_and_model

In [31]:
!ls saved

MultiVAE-Dec-20-2023_11-11-26.pth


In [34]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
        model_file="/content/saved/MultiVAE-Dec-20-2023_11-11-26.pth",
    )

Max value of user's history interaction records has reached 20.9471766848816% of the total.


In [35]:
model

MultiVAE(
  (encoder): Sequential(
    (0): Linear(in_features=3294, out_features=600, bias=True)
    (1): Tanh()
    (2): Linear(in_features=600, out_features=128, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=600, bias=True)
    (1): Tanh()
    (2): Linear(in_features=600, out_features=3294, bias=True)
  )
)

In [39]:
test_data

In [54]:
from recbole.utils.case_study import full_sort_topk, full_sort_scores
topk_score, topk_iid_list = full_sort_topk(
        uid_series, model, test_data, k=10, device=config["device"]
    )

IndexError: ignored

In [51]:
print(topk_score)  # scores of top 10 items
print(topk_iid_list)  # internal id of top 10 items
external_item_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu())
print(external_item_list)  # external tokens of top 10 items
print()

tensor([[3.2173, 3.0331, 3.0272, 2.9966, 2.9808, 2.9379, 2.9001, 2.8665, 2.8302,
         2.8113],
        [3.8060, 3.6202, 3.4049, 3.3864, 3.3722, 3.3667, 3.3532, 3.2788, 3.2745,
         3.2004]], device='cuda:0')
tensor([[  73,  338,   22,  350,  842,  989,  293,  705,  482,  411],
        [  87,   57,  383,  796,  725, 1485,  707,  415,  900,  846]],
       device='cuda:0')
[['13865' '1785' '10440' '4436' '12173' '11754' '3734' '11310' '14317'
  '5693']
 ['13545' '16166' '13159' '5411' '4718' '14942' '10761' '13018' '11749'
  '5051']]



In [45]:
print(external_item_list[0])

['13865' '1785' '10440' '4436' '12173' '11754' '3734' '11310' '14317'
 '5693']
